# 베이지안 옵티마이징 (Bayesian Optimization)
- 기존 하이퍼 파라미터 튜닝 방법인 Grid Search, Random Search의 문제점 : 최적의 값을 찾아갈 수 없음
- 개념 : 
    "Gausain Process"라는 통계학을 기반으로 만들어진 모델로, 
    여러개의 하이퍼 파라미터들에 대해서,
    "Aqusition Fucntion"을 적용했을 때,
    "가장 큰 값"이 나올 확률이 높은 지점을 찾아냅니다.
- Bayesian Optimization 패키지 단계 : 
    1. 변경할 하이퍼 파라미터의 범위 설정
    2. Bayesian Optimization 패키지를 통해, 하이퍼 파라미터의 범위 속 값들을 랜덤하게 가져온다.
    3. 처음 **R번**은 정말 **Random**하게 좌표를 꺼내 **성능**을 확인한다.
    4. 이후 **B번**은 Bayesian Optimization을 통해 **B번만큼 최적의 값**을 찾는다. 
- 아나콘다 베이지안 옵티미제이션 설치 : 
    - `conda install bayesian-optimization`
- bayes_opt 배키지에서 BayesianOptimization 불러오기
    - `from bayes_opt import BayesianOptimization`

# 하이퍼 파라미터 3가지 튜닝 방법 비교하기
### Grid Search vs Random Search vs Bayesian Optimization
1. Grid Search (그리드 서치)
- 기법 :  Grid Search는 **사전에 탐색할 값**들을 미리 지정해주고, 그 **값**들의 모든 조합을 바탕으로 성능의 최고점을 찾아냅니다.
- 장점 : 내가 원하는 범위를 정확하게 비교 분석이 가능하다.
- 단점 : 
    - **시간**이 오래걸림. (4개의 파라미터에 대해서, 4가지 값들을 지정해두고, 한 번 탐색하는데 1분이 걸린다면 -> 4*4*1분 = 16분 소요)
    - 정확도 ⬇️ : 성능의 최고점이 아닐 가능성이 높음
    - 최적화 검색 방식 : 여러개들을 비교 분석해서 최고를 찾아내는 기법
        - 참고 : "최적화 탐색"(성능이 가장 높은 점으로 점차 찾아가는 기법)

2. Random Search (랜덤 서치)
- 기법 : **사전에 탐색할 값들의 범위**를 지정해주고, 그 **범위** 속에서 가능한 조합을 바탕으로 최고점을 찾아냄
- 장점 :
    - 소요 시간 ⬇️ : Grid Search 보다 시간이 짧게 걸림
    - 정확도 ⬆️ : Grid Search 보다, 랜덤하게 점을 찍으므로, 성능 ⬆️ 가능성 O
- 단점 :
    - 정확도 ⬇️ : Grid Search 보다, 랜덤하게 점을 찍으므로, 성능 ⬇️ 가능성 O
    - 가변성 : 하이퍼 파라미터 범위가 넓거나, 사용할 때마다 값이 달라지기 때문에 seed 고정 필수
    - 최적화 검색 방식 : 여러개들을 비교 분석해서 최고를 찾아내는 기법

3. Bayesian Optimization (베이지안 최적화)
- 기법 : **하이퍼파라미터의 범위**를 지정한 후, **Random하게 R 번 탐색한 후, B번 만큼 최적의 값**을 찾아간다.
- 장점 : 
    - 정말 "최적의 값"을 찾아갈 수 있다.
    - 상대적으로 시간이 덜 걸린다.
    - 엔지니어가 그 결과값을 신뢰할 수 있다.
- 단점 :
    - 소요 시간 ⬆️ : 랜덤하게 찍은 값이 달라질 경우, 최적화 하는데 오래 걸릴 수 있음
    - 랜덤하게 찍은 값이 ⬇️ 일 때 : 최적의 값을 탐색하는게 불가능
    - 랜덤하게 찍은 값이 ⬆️ 일 때 : 최적화 이전에 이미 최적값을 보유할 수도 있음

# Train vs Validation vs Test
* Train
    - **모델을 학습하기 위한 데이터셋**으로 이때 학습은 최적의 파라미터를 찾는 것입니다.
    - 즉, Train data는 오직 **학습**을 위한 데이터셋 
 
* Validation
    - 학습이 이미 **완료된 모델을 검증**하기 위한 데이터셋
    - 학습이 된 여러 가지 모델 중 가장 좋은 하나의 모델을 고르기 위한 데이터셋
    - 학습 과정에 직접적인 관여 ❌

* test
    - 모델의 **최종 성능**을 평가하기 위한 데이터셋 
    - 학습 과정에 관여 ❌

# Bayesian Optimization(베이지안 최적화) 실습
1. 학슬할 데이터 X, 목표변수 y 준비
    ```
    X = train.drop(columns = ['index', 'quality'])
    y = train['quality]
    ```
2. 랜덤포레스트의 하이퍼 파라미터 범위를 딕셔너리 형태로 저장
* 딕셔너리의 Key는 랜덤포레스트의 하이퍼파라미터, value는 탐색할 하이퍼파라미터 범위 값임
    ```
    rf_parameter_bounds = {
        'max_depth' : (1,3) #나무 깊이 범위
        'n_estimators' : (30, 100) #모델 개수 범위
    }
    ```
3. 함수 구현
    1. 모델 사용 함수 구현
        ```
        def rf_bo(max_depth, n_estimators): #함수에 들어가는 인자 = 딕셔너리 Key
        rf_params = {
            'max_depth' : int(round(max_depth)),
            'n_estimators' : int(round(n_estimators)),      
        } #함수 인자를 통해 새롭게 하이퍼파라미터 딕셔너리 생성
        rf = RandomForestClassifier(**rf_params) #하이퍼 딕셔너리 바탕의 모델 생성
        X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size = 0.2, ) #train_test_split() 을 통해 데이터 Train-Valid 나누기
        rf.fit(X_train,y_train) #모델 학습
        score = accuracy_score(y_valid, rf.predict(X_valid)) #모델 성능 측정 
        return score #모델 점수 반환
        ```
    2. Bayesian Optimization 사용하기
        ```
        BO_rf = BayesianOptimization(f = rf_bo, pbounds = rf_parameter_bounds,random_state = 0) #베이지안 최적화 모델 저장
        BO_rf.maximize(init_points = 5, n_iter = 5) #베이지안 최적화 실행 #init_points = 임의로 찍을 초기값 개수 #n_iters = 파라미터 검색횟수
        max_params = BO_rf.max['params'] #하이퍼파라미터의 결과값 불러오기 #"max_params"라는 변수에 저장
        max_params['max_depth'] = int(max_params['max_depth']) 
        max_params['n_estimators'] = int(max_params['n_estimators'])  
        BO_tuend_rf = RandomForestClassifier(**max_params) # 튜닝 결과 "BO_tuend_rf"라는 변수에 저장

        ```



    